Helper Methods

In [1]:
def check_for_correct_residue(x):
    if x[0].isupper() and x[-1].isupper() and x[1:-1].isdigit():
        return True
    else:
        return False
def combine(x):
    return str(x[0]) + "_" + str(x[1])
def combine_index(x):
    return str(x[0]) + "_" + str(x[1][1:-1])
def synonymous(x):
    if (x[0] == x[-1:]) or (x[0] == '*') or (x[-1] == '*') or (x[0] == '_') or (x[-1] == '_'):
        return False
    else:
        return True
def get_index(x):
    return x[1:-1]
def get_gene(x):
    return x.split('_')[0]
def get_position(x):
    return x.split('_')[1][1:-1]

In [ ]:
def PHBR(x):
    number_of_alleles = len(x)
    s = 0
    for i in range(number_of_alleles):
        s += 1/float(x[i])
    return number_of_alleles / float(s)

Gather protein sequences

In [2]:
def get_length(x):
    return len(x)

In [5]:
def gather_protein_sequences():
    # Ensemble (with sequences)
    proteins, genes, sequences = [], [], []
    fasta_sequences = SeqIO.parse(open("/cellar/users/ramarty/Data/hla_ii/references/Homo_sapiens.GRCh38.pep.all.fa"),'fasta')
    for fasta in fasta_sequences:
        gene, protein, sequence = fasta.description.split('gene_symbol:')[1].split(' ')[0], fasta.id, fasta.seq.tostring()
        proteins.append(protein)
        genes.append(gene)
        sequences.append(sequence)
    ensemble = pd.DataFrame({'protein': proteins, 'gene': genes, 'sequence': sequences})
    ensemble['protein'] = ensemble['protein'].map(lambda x: str(x)[:-2])

    # Uniprot (mapping)
    uniprot_import = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/references/HUMAN_9606_idmapping_selected.tab', sep='\t', header=None)
    proteins, uniprot_ACs = [], []
    for i in range(len(uniprot_import)):
        try:
            proteins.append(uniprot_import.ix[i, 20].split(';')[0])
            uniprot_ACs.append(uniprot_import.ix[i, 0])
        except:
            None
    uniprot = pd.DataFrame({'protein': proteins, 'uniprot_AC': uniprot_ACs})
    # Drop all rows without a Uniprot AC
    merged = pd.merge(ensemble, uniprot, on='protein', how='left')
    merged = merged.drop_duplicates()
    merged = merged.dropna()
    # Merge to make single, connonical entry per gene
    merged['length'] = merged['sequence'].apply(get_length)
    merged = merged.sort_values('length', ascending=False)
    merged = merged.drop_duplicates(subset='protein')
    return merged

In [9]:
def gather_protein_sequences_all():
    # Ensemble (with sequences)
    proteins, genes, sequences = [], [], []
    fasta_sequences = SeqIO.parse(open("/cellar/users/ramarty/Data/hla_ii/references/Homo_sapiens.GRCh38.pep.all.fa"),'fasta')
    for fasta in fasta_sequences:
        gene, protein, sequence = fasta.description.split('gene_symbol:')[1].split(' ')[0], fasta.id, fasta.seq.tostring()
        proteins.append(protein)
        genes.append(gene)
        sequences.append(sequence)
    ensemble = pd.DataFrame({'protein': proteins, 'gene': genes, 'sequence': sequences})
    ensemble['protein'] = ensemble['protein'].map(lambda x: str(x)[:-2])
    # Add missing gene
    ensemble.loc[len(ensemble)+1] = ['CRIPAK', '_', 'MHEPSLCANVECPPAHTCPCGVPACSCAHVECPPAHTCRCGVPACSHMPMWSARLLTRAHVECPPAHTRVHVECPPAHVPMWSAHLLTCADVECHLLTHVPMWSARLLTCPCGVPACSHVPMRSARLLTRAHAECPPAHTCPCGVPACSHVPMRSARLLTRADVECPPAHTCPCGVPACSHVPTWSARLITRAHVECSPAHTCRCGVPACSHVPMWSVRLLTRADAECPPAHTCRCGVPACSHVPMWSARLLTCRCGVPACSHVPMWSARLLTCRCGVPACSHVPMWSARLLTRAHVECPPAHTCRRGVPACSRAHMECPPAHTCHCGVPACSHTCRCGVPACSHVPMWSARLLTRAHVECPPAHTRAHVECPPAHTCPCGVPACSHTCPCGVPACSHKALAWWFCRFPVLPAESDAVTVHSTHGGFLIRFYVKDPFYISLHLEIT']
    return ensemble

Create the fasta files

In [2]:
def create_fasta_files(gene_sequence_df, category, mutations, mhc_class, status):
    # put mutations in dataframe
    success = []
    gene, residue = [], []
    for mutation in mutations:
        c = mutation.split('_')
        gene.append(c[0])
        residue.append(c[1])
    gene_residue = pd.DataFrame({'gene': gene, 'residue': residue})

    # get format with gene, residue, sequence
    combined = pd.merge(gene_residue, gene_sequence_df, on='gene', how='left')
    combined = combined[['gene', 'residue', 'sequence']]

    # check that all mutations are in table
    if len(gene_residue) > len(combined):
        print "Error: missing sequences for residues", len(gene_residue) - len(combined)

    # create fasta files
    for row in combined.iterrows():
        output = fasta_machinery(row, category, mhc_class, status)
        success.append(output)
    return success

In [1]:
def fasta_machinery(row, category, mhc_class, status):
    if mhc_class == 'class_i':
        y = 10
    else:
        # double check this before I run mhc-ii
        y = 14
    gene = row[1]['gene']
    residue = row[1]['residue']
    sequence = str(row[1]['sequence'])
    position = int(residue[1:len(residue)-1]) -1
    old_aa = residue[0]
    new_aa = residue[-1:]
    
    if len(sequence) >= position and sequence[position] == old_aa: #(not accurate for random)
        mutated_sequence = sequence[:position] + new_aa + sequence[position+1:]
        output_f = open('/cellar/users/ramarty/Data/hla/residue/fasta_files/{0}/{1}/{2}/{3}_{4}.fa'.format(status, mhc_class, category, gene, residue), 'w')
        
        # specify start and end
        if (position-1) - y >= 0:
            start = (position-1) - y 
        else:
            start = -1
            
        if (position-1) + y <= len(sequence)-1:
            end = (position-1) + y
        else:
            end = len(sequence) - 2

        output_f.write('>gi {0}_{1}\n'.format(gene, residue))
        if status == 'mut':
            output_f.write(mutated_sequence[start+1:end+2])
        else:
            output_f.write(sequence[start+1:end+2])
        output_f.close()
        return True

    else:
        print "Error: sequence is not long enough or not the same background", gene, residue
        return False

Pathogens

In [1]:
def update_viral(f, kind):
    try:
        gene = f.split('.')[0]
        sequence = open('/cellar/users/ramarty/Data/nrnb01_nobackup/hla/fasta_files/{0}/{1}.i.fa'.format(kind, f)).readlines()[1]

        possible_seqs = list(virus[virus.Entry == gene].Sequence)

        for seq in possible_seqs:
            if sequence in seq:
                position = seq.find(sequence)
                if position >= 15:
                    new_sequence = seq[position-2:position+27]
                else:
                    new_sequence = list(virus.sample().Sequence)[0][:29]
                    position = 15
                #print len(new_sequence)
                break

        output_f = open('/cellar/users/ramarty/Data/hla/residue/fasta_files/mut/class_i/{0}/{1}.fa'.format(kind, gene), 'w')
        output_f.write('>gi {0}_{1}\n'.format(gene, position))
        output_f.write(new_sequence)
        output_f.close()
    except:
        
        new_sequence = list(virus.sample().Sequence)[0][:21]
        position = 11
        
        output_f = open('/cellar/users/ramarty/Data/hla/residue/fasta_files/mut/class_i/{0}/{1}.fa'.format(kind, gene), 'w')
        output_f.write('>gi {0}_{1}\n'.format(gene, position))
        output_f.write(new_sequence)
        output_f.close()
        

In [2]:
def update_bacterial(f, kind):
    try:
        gene = f.split('.')[0]
        sequence = open('/cellar/users/ramarty/Data/nrnb01_nobackup/hla/fasta_files/{0}/{1}.i.fa'.format(kind, f)).readlines()[1]

        possible_seqs = list(bacteria[bacteria.Entry == gene].Sequence)

        for seq in possible_seqs:
            if sequence in seq:
                position = seq.find(sequence)
                if position >= 11:
                    new_sequence = seq[position-2:position+19]
                else:
                    new_sequence = seq[:21]
                    position = 11
                #print len(new_sequence)
                break

        output_f = open('/cellar/users/ramarty/Data/hla/residue/fasta_files/mut/class_i/{0}/{1}.fa'.format(kind, gene), 'w')
        output_f.write('>gi {0}_{1}\n'.format(gene, position))
        output_f.write(new_sequence)
        output_f.close()
    except:
        new_sequence = list(bacteria.sample().Sequence)[0][:21]
        position = 11
        
        output_f = open('/cellar/users/ramarty/Data/hla/residue/fasta_files/mut/class_i/{0}/{1}.fa'.format(kind, gene), 'w')
        output_f.write('>gi {0}_{1}\n'.format(gene, position))
        output_f.write(new_sequence)
        output_f.close()